### Spark Initialization

In [1]:
import findspark
findspark.init()

In [2]:
import sys
import copy
import csv

from string import atoi
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
import numpy as np

In [4]:
conf = SparkConf().setAppName("ContentBased")
conf = conf.setMaster("local[*]")

In [5]:
sc  = SparkContext(conf=conf)

### Load Train and Test Data

In [15]:
trainData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)
testData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_test.txt",use_unicode=False)

In [16]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

In [17]:
avg_rating = train_rdd.map(lambda x: (x[0][0], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))
prod_rating = train_rdd.map(lambda x: (x[0][1], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))

In [18]:
avg_rating = avg_rating.map(lambda x: ((x[0]), x[1]))

In [19]:
avg_rating.take(5)

[('AKqMnr1bEFIZDG_zQzX8sA', 3.45),
 ('S1cjSFKcS5NVc3o1MkfpwA', 3.574074074074074),
 ('bSvNU2vABlaBi1ooF4KNJg', 3.3684210526315788),
 ('8ye6-7YYMsp0YizCLyrxcA', 4.184210526315789),
 ('oIN1CjzbCCol4kPueU9WzQ', 3.5555555555555554)]

In [20]:
train_temp = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), 1))

### Load review data

In [21]:
data = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)

In [22]:
data.take(6)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

In [23]:
train_data = data.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), x[7])).join(train_temp)

In [24]:
train_data = train_data.map(lambda x: (x[0][0], x[0][1], x[1][0]))

### Collect user data from train data (User, Review)

In [25]:
userReview = train_data.map(lambda x: (x[0], x[2])).groupByKey().mapValues(list)

In [26]:
userReviewCollected = userReview.map(lambda x: (x[0], x[1][0].split(" ")))

In [27]:
userReviewCollected.take(5)

[('AKqMnr1bEFIZDG_zQzX8sA',
  ['place',
   'breathe',
   'fresh',
   'air',
   'find',
   'hot',
   'dog',
   'fantastic',
   'buns',
   'toppings',
   'fresh',
   'homemade',
   'fresh',
   'cut',
   'french',
   'fry',
   'hot',
   'crispy',
   'staff',
   'pleasant',
   'friendly',
   'chicago',
   'dog',
   'plain',
   'dog',
   'relish',
   'ketchup',
   'right',
   'point',
   'fry',
   'yummmmmm',
   'little',
   'small',
   'inside',
   'serve',
   'purpose',
   'mood',
   'fresh',
   'fry',
   'great',
   'hot',
   'dog',
   'get',
   'sure',
   'definitely',
   'go',
   'back']),
 ('hg71CGHy9bwpgY8cGVVg4w',
  ['wife',
   'come',
   'lunch',
   'walk',
   'buy',
   'chocolate',
   'moose',
   'look',
   'say',
   'chocolate',
   'get',
   'ta',
   'go',
   'store',
   'huge',
   'sort',
   'candy',
   'handmade',
   'huge',
   'selection',
   'kinds',
   'great',
   'totally',
   'delicious',
   'truffles',
   'order',
   '15',
   'different',
   'truffles',
   'everyone',
   

### Collect product data from train data (Product, Review)

In [28]:
prodReview = train_data.map(lambda x: (x[1], x[2])).groupByKey().mapValues(list)

In [29]:
prodReviewCollected = prodReview.map(lambda x: (x[0], x[1][0].split(" ")))

In [30]:
prodReviewCollected.take(5)

[('5REYrZfsX3m4E3FTwovp5Q',
  ['try',
   'first',
   'time',
   'last',
   'night',
   'pretty',
   'good',
   'one',
   'thing',
   'really',
   'annoy',
   'restaurant',
   'see',
   'review',
   'verde',
   'entire',
   'restaurant',
   'fill',
   'reservations',
   'can',
   'not',
   'even',
   'seat',
   'walk',
   '2',
   'people',
   'restaurant',
   'empty',
   'really',
   'avocado',
   'corn',
   'ceviche',
   'mojo',
   'criollo',
   'nigiri',
   'avocado',
   'crab',
   'delicious',
   'mojo',
   'criollo',
   'favorite',
   'everything',
   'taste',
   'super',
   'fresh',
   'flavorful',
   'serve',
   'good',
   'size',
   'date',
   'share',
   'wine',
   'meh',
   'would',
   'think',
   'place',
   'thats',
   'like',
   'tapas',
   'wine',
   'bar',
   'wine',
   'would',
   'better',
   'order',
   'frontera',
   'sauvignon',
   'blanc',
   'often',
   'buy',
   'liquor',
   'store',
   'slo',
   'wine',
   'hm',
   'lame',
   'hate',
   'pay',
   '9',
   'glass',


### Doc2Vec model creation

In [31]:
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

/Users/lakshya/anaconda/lib/python2.7/site-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


Convert user and product rdd to pandas dataframe

In [33]:
spark = SparkSession(sc)

userPandas = userReviewCollected.toDF().toPandas()
prodPandas = prodReviewCollected.toDF().toPandas()

Create Doc2Vec model on product review text

In [34]:
documents_prod = [TaggedDocument(doc, [i]) for i, doc in enumerate(prodPandas['_2'])]

In [35]:
documents_prod

[TaggedDocument(words=[u'try', u'first', u'time', u'last', u'night', u'pretty', u'good', u'one', u'thing', u'really', u'annoy', u'restaurant', u'see', u'review', u'verde', u'entire', u'restaurant', u'fill', u'reservations', u'can', u'not', u'even', u'seat', u'walk', u'2', u'people', u'restaurant', u'empty', u'really', u'avocado', u'corn', u'ceviche', u'mojo', u'criollo', u'nigiri', u'avocado', u'crab', u'delicious', u'mojo', u'criollo', u'favorite', u'everything', u'taste', u'super', u'fresh', u'flavorful', u'serve', u'good', u'size', u'date', u'share', u'wine', u'meh', u'would', u'think', u'place', u'thats', u'like', u'tapas', u'wine', u'bar', u'wine', u'would', u'better', u'order', u'frontera', u'sauvignon', u'blanc', u'often', u'buy', u'liquor', u'store', u'slo', u'wine', u'hm', u'lame', u'hate', u'pay', u'9', u'glass', u'wine', u'restaurant', u'easily', u'buy', u'10', u'liquor', u'store', u'seem', u'justify', u'least', u'slo', u'bottle', u'service', u'good', u'nice', u'time', u'wou

Create Doc2Vec model on user review text

In [36]:
documents_user = [TaggedDocument(doc, [i+len(documents_prod)]) for i, doc in enumerate(userPandas['_2'])]

In [37]:
documents_user

[TaggedDocument(words=[u'place', u'breathe', u'fresh', u'air', u'find', u'hot', u'dog', u'fantastic', u'buns', u'toppings', u'fresh', u'homemade', u'fresh', u'cut', u'french', u'fry', u'hot', u'crispy', u'staff', u'pleasant', u'friendly', u'chicago', u'dog', u'plain', u'dog', u'relish', u'ketchup', u'right', u'point', u'fry', u'yummmmmm', u'little', u'small', u'inside', u'serve', u'purpose', u'mood', u'fresh', u'fry', u'great', u'hot', u'dog', u'get', u'sure', u'definitely', u'go', u'back'], tags=[3098]),
 TaggedDocument(words=[u'wife', u'come', u'lunch', u'walk', u'buy', u'chocolate', u'moose', u'look', u'say', u'chocolate', u'get', u'ta', u'go', u'store', u'huge', u'sort', u'candy', u'handmade', u'huge', u'selection', u'kinds', u'great', u'totally', u'delicious', u'truffles', u'order', u'15', u'different', u'truffles', u'everyone', u'totally', u'delicious', u'would', u'definitely', u'go', u'also', u'would', u'highly', u'recommend', u'place', u'need', u'sugar', u'candy', u'rush'], tag

Combine both user an

In [38]:
documents = documents_prod+documents_user

In [39]:
model = Doc2Vec(documents, size = 3000, min_count = 1, workers=4)

/Users/lakshya/anaconda/lib/python2.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [40]:
feature_vectors = []
for i in range(0, len(documents_prod)):
    feature_vectors.append(model.docvecs[i])

In [41]:
feature_vectors[0]

array([ 0.0174081 , -0.00577185, -0.01162341, ..., -0.00172507,
       -0.00187236,  0.01294933], dtype=float32)

Add Doc2Vec vectors to product dataframe

In [43]:
prodPandas['Vector'] = feature_vectors

In [44]:
prodPandas

,_1,_2,Vector
0,5REYrZfsX3m4E3FTwovp5Q,"[try, first, time, last, night, pretty, good, ...","[0.0174081, -0.00577185, -0.0116234, -0.006113..."
1,HWrbZS1mxVRj2Y2VwMmDMg,"[oh, man, word, can, not, describe, excite, ba...","[0.0216439, -0.00462718, -0.0173346, -0.007567..."
2,MvlQo4bev1eqp1q0HYOLHg,"[first, dance, class, probably, decade, probab...","[0.00531396, -0.0021056, -0.00318673, -0.00173..."
3,X9Bql7RrPU5Mab5-hJsI8A,"[2nd, visit, promise, first, time, order, feat...","[0.0120019, -0.000385666, -0.0113439, -0.00413..."
4,owO2UkNKk9qrWWd_PTYLDA,"[feel, like, ive, random, experience, place, r...","[0.0357995, -0.0063142, -0.0298217, -0.0124923..."
5,rc9SfYli96cnlPDMyJ7hyA,"[go, lunch, friends, group, 7, round, table, b...","[0.00558426, 0.000327907, -0.00573921, -0.0018..."
6,94VxE6XLTrXmz4yXWEJbAg,"[come, instead, ds, need, buy, case, beer, the...","[0.00501451, -0.00120674, -0.00392044, -0.0016..."
7,anfgvbiobw4M0Wnvx_os3Q,"[its, good, place, try, ive, never, wow, prett...","[0.0115027, -0.00198811, -0.00907118, -0.00352..."
8,vLGYQ96AjKV5zpW0q3795g,"[breakfast, pizza, checkgood, pizza, accent, e...","[0.00933751, -0.00149994, -0.00754559, -0.0028..."
9,7l-CCyNnvT86GMR5EtYjog,"[jamaican, tacos, tonight, die, cant, speak, a...","[0.00645934, -0.0019303, -0.00462884, -0.00226..."


Add Doc2Vec vectors to user dataframe

In [45]:
feature_vectors = []
for i in range(len(documents_prod), len(documents_prod)+len(documents_user)):
    feature_vectors.append(model.docvecs[i])

In [46]:
userPandas['Vector'] = feature_vectors

In [47]:
userPandas

,_1,_2,Vector
0,AKqMnr1bEFIZDG_zQzX8sA,"[place, breathe, fresh, air, find, hot, dog, f...","[0.0149788, -0.00195833, -0.0130463, -0.004877..."
1,hg71CGHy9bwpgY8cGVVg4w,"[wife, come, lunch, walk, buy, chocolate, moos...","[0.00458378, -0.00100534, -0.00332994, -0.0013..."
2,S1cjSFKcS5NVc3o1MkfpwA,"[would, think, vegan, friendly, hipster, resta...","[0.0101831, -0.0035301, -0.00649505, -0.003455..."
3,eYnS0WVYYykN_gRuMUzoOw,"[stop, pittsburgh, home, garden, show, quick, ...","[0.00783149, -0.00560254, -0.0025686, -0.00320..."
4,A0ssDikuj0SQr9mVqnh91Q,"[sit, bar, good, drink, bunch, beer, options, ...","[0.00296203, -0.001174, -0.00157797, -0.001017..."
5,TDOF4SZAh7MsFgfshOL66A,"[three, days, later, neighbor, still, texting,...","[0.0338488, -0.00862429, -0.025496, -0.0121223..."
6,MLf4JUa1uzQkDLrBJ5yJHA,"[come, group, end, order, regular, size, pho, ...","[0.0078924, -0.00070032, -0.00698314, -0.00246..."
7,6BnvEluqfazShd1tg9kqGg,"[pleasantly, surprise, tequila, cowboy, never,...","[0.00993479, -0.00211394, -0.00728097, -0.0033..."
8,DYqujEDnSMgwL7sFTRSv8w,"[tasty, lunch, less, 10, close, universities, ...","[0.0288285, -0.00824692, -0.0207248, -0.010113..."
9,zg6LAy9gGLVH9JtcYP9gJg,"[5, star, experience, anyone, know, know, love...","[0.0147838, -0.00410555, -0.0106086, -0.005274..."


Review text not needed

In [48]:
del prodPandas['_2']
del userPandas['_2']

### Create user numpy matrix from feature vectors

In [49]:
user_matrix = np.zeros((len(userPandas), len(feature_vectors[0])))
idx = 0
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector'])
    user_matrix[idx] = vector
    idx += 1

In [50]:
user_matrix

array([[  1.49787962e-02,  -1.95832923e-03,  -1.30463475e-02, ...,
         -6.94139104e-04,   2.92307744e-03,   9.58795566e-03],
       [  4.58377600e-03,  -1.00533885e-03,  -3.32994387e-03, ...,
         -2.92382407e-04,   1.48732670e-05,   3.65400664e-03],
       [  1.01831248e-02,  -3.53010138e-03,  -6.49504736e-03, ...,
         -1.05190906e-03,  -1.72721490e-03,   7.80070247e-03],
       ..., 
       [  1.67488884e-02,  -4.65128245e-03,  -1.22928321e-02, ...,
         -1.77616987e-03,  -1.78476737e-04,   1.28231868e-02],
       [  8.81074183e-03,  -3.35345231e-03,  -5.47186099e-03, ...,
         -9.22275649e-04,  -1.77641155e-03,   6.73798891e-03],
       [  2.27717031e-03,  -5.28912875e-04,  -1.59447978e-03, ...,
         -2.06869416e-04,   4.48589199e-05,   1.81625481e-03]])

### Create product numpy matrix from feature vectors

In [51]:
prod_matrix = np.zeros((len(prodPandas), len(feature_vectors[0])))
idx = 0
for index, row in prodPandas.iterrows():
    vector = np.array(row['Vector'])
    prod_matrix[idx] = vector
    idx += 1

In [52]:
prod_matrix

array([[ 0.0174081 , -0.00577185, -0.01162341, ..., -0.00172507,
        -0.00187236,  0.01294933],
       [ 0.02164385, -0.00462718, -0.01733457, ..., -0.00181533,
         0.00195026,  0.01563367],
       [ 0.00531396, -0.0021056 , -0.00318673, ..., -0.00065144,
        -0.00117965,  0.00447223],
       ..., 
       [ 0.00957174, -0.00295554, -0.00643366, ..., -0.00082952,
        -0.000999  ,  0.00654024],
       [ 0.00614555, -0.00121817, -0.00478792, ..., -0.00064255,
         0.00082209,  0.00472452],
       [ 0.00933781, -0.0026564 , -0.006993  , ..., -0.00090229,
        -0.00104649,  0.0059106 ]])

### Compute cosine similarity by taking dot product

In [53]:
similarity_matrix = np.dot(user_matrix, prod_matrix.T)

In [54]:
similarity_matrix.shape

(987, 3098)

### Flatten similarity matrix to relate with user and products

In [55]:
prod = prodPandas['_1'].values
user = userPandas['_1'].values

In [56]:
zf = similarity_matrix.flatten()
xr = np.repeat(user, prod.size)
yt = np.tile(prod, user.size)
d = np.stack((xr, yt, zf), axis=-1)

In [57]:
d.shape

(3057726, 3)

### Convert similarity matrix to RDD

In [58]:
similarity_rdd = sc.parallelize(d)

In [59]:
similarity_rdd.take(5)

[array([u'AKqMnr1bEFIZDG_zQzX8sA', u'5REYrZfsX3m4E3FTwovp5Q',
        0.11769174652622787], dtype=object),
 array([u'AKqMnr1bEFIZDG_zQzX8sA', u'HWrbZS1mxVRj2Y2VwMmDMg',
        0.14141611749972535], dtype=object),
 array([u'AKqMnr1bEFIZDG_zQzX8sA', u'MvlQo4bev1eqp1q0HYOLHg',
        0.03801341828643435], dtype=object),
 array([u'AKqMnr1bEFIZDG_zQzX8sA', u'X9Bql7RrPU5Mab5-hJsI8A',
        0.07441758639759136], dtype=object),
 array([u'AKqMnr1bEFIZDG_zQzX8sA', u'owO2UkNKk9qrWWd_PTYLDA',
        0.2354921480774784], dtype=object)]

### Use Train and Test Data for predictions

In [60]:
trainData.take(5)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

Create key on (user, product)

In [61]:
similarity_rdd = similarity_rdd.map(lambda x: ((x[0], x[1]), float(x[2])))

In [62]:
similarity_rdd.take(5)

[((u'AKqMnr1bEFIZDG_zQzX8sA', u'5REYrZfsX3m4E3FTwovp5Q'), 0.11769174652622787),
 ((u'AKqMnr1bEFIZDG_zQzX8sA', u'HWrbZS1mxVRj2Y2VwMmDMg'), 0.14141611749972535),
 ((u'AKqMnr1bEFIZDG_zQzX8sA', u'MvlQo4bev1eqp1q0HYOLHg'), 0.03801341828643435),
 ((u'AKqMnr1bEFIZDG_zQzX8sA', u'X9Bql7RrPU5Mab5-hJsI8A'), 0.07441758639759136),
 ((u'AKqMnr1bEFIZDG_zQzX8sA', u'owO2UkNKk9qrWWd_PTYLDA'), 0.2354921480774784)]

Join similarity matrix with train and test RDD to take only similarity values for training and testing

In [63]:
train = similarity_rdd.join(train_rdd)
test = similarity_rdd.join(test_rdd)

In [64]:
train.take(5)

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (0.2963564521319387, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (0.13118959472743053, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (0.5975845360218869, 3.0)),
 ((u'M9MXoSsb193m1g0QmmUYBQ', u'-i3pCgQi_Y9NiSSWs6G7bw'),
  (0.6600529920408007, 5.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'x8WI_GkeGHGJCXggDm8flg'),
  (0.05050062491191396, 3.0))]

Convert RDD to List

In [65]:
train_ratings = train.collect()
test_ratings = test.collect()

In [66]:
test_rdd.take(5)

[(('JiPMk9WmbJu-VfTRAKpZpw', 'PdDpIGwBZoTYzOVasT-WuA'), 4.0),
 (('2wKnvn68eWybc7ID-7UQmQ', 'khRo2a5OaIjumox-tkg3GA'), 4.0),
 (('LsWpfxWjLQcazDqnZ_A62g', 'D_pwairtGGR0V_w2xx5XeA'), 2.0),
 (('0N9bSCmoJMoGmR0EldzjQg', '3iaOYhNoc6XL935MqnxJSQ'), 5.0),
 (('-hietrA8M58asfpyJkCyiA', 'O1ird5yRyuDFnOmYu90OoA'), 4.0)]

In [67]:
train_ratings

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (0.2963564521319387, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (0.13118959472743053, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (0.5975845360218869, 3.0)),
 ((u'M9MXoSsb193m1g0QmmUYBQ', u'-i3pCgQi_Y9NiSSWs6G7bw'),
  (0.6600529920408007, 5.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'x8WI_GkeGHGJCXggDm8flg'),
  (0.05050062491191396, 3.0)),
 ((u'135DbbQnr3BEkQbBzZ9T1A', u'CTiWs36A_sX3eahdqrYxUQ'),
  (1.4459905012574903, 5.0)),
 ((u'rF-MrPztxHEm6KJe0JUJIQ', u'w2tr0PA0b3tg8pId0kjM-A'),
  (0.04895277285174934, 3.0)),
 ((u'vHHjaq9pSuwq8uCrQ85qlw', u'_R1jBQQieKpNGMBqmrLRyA'),
  (0.22644975660057837, 2.0)),
 ((u'0N9bSCmoJMoGmR0EldzjQg', u'F9IJ3EddoyW89O0DFikPTQ'),
  (0.05066763848772042, 1.0)),
 ((u'1ZPnQs-tdvbX8ROjtnzcEg', u'OXrFWgoz533T8tMRemkiww'),
  (0.0010334827410670587, 5.0)),
 ((u'PZNMPWCViVX8JLsn10MSnQ', u'MNmQ1s-zdjEgm3bdRs7iEw'),
  (0.06294436673183089, 4.0)),
 ((u'tR12WeWha2DGhUrKcv

In [68]:
train.take(5)

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (0.2963564521319387, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (0.13118959472743053, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (0.5975845360218869, 3.0)),
 ((u'M9MXoSsb193m1g0QmmUYBQ', u'-i3pCgQi_Y9NiSSWs6G7bw'),
  (0.6600529920408007, 5.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'x8WI_GkeGHGJCXggDm8flg'),
  (0.05050062491191396, 3.0))]

### Training Regressor on similarity values

In [69]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

/Users/lakshya/anaconda/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Convert data to numpy array for regressor

In [70]:
X_train = []
y_train = []
for ratings in train_ratings:
    X_train.append(ratings[1][0])
    y_train.append(ratings[1][1])

X_train = np.array(X_train)
X_train = X_train.reshape(-1,1)

In [71]:
X_test = []
y_test = []
for ratings in test_ratings:
    X_test.append(ratings[1][0])
    y_test.append(ratings[1][1])

X_test = np.array(X_test)
X_test = X_test.reshape(-1,1)

In [72]:
X_test

array([[ 0.00764388],
       [ 0.01788623],
       [ 0.03135834],
       ..., 
       [ 0.00922155],
       [ 0.2620674 ],
       [ 0.04164859]])

Train regressor on training data and make prediction on Test data<br>
Computed Mean Squared Error on predicted values

In [88]:
forest = RandomForestRegressor(max_depth=10, n_estimators=20)

rs = GradientBoostingRegressor(loss='ls', learning_rate=0.005, n_estimators=5)

reg = LinearRegression()

rs.fit(X_train, y_train)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = rs.predict(X_test)

train_expected = y_train
train_predicted = rs.predict(X_train)

print("Training:\n%s" % np.sqrt(mean_squared_error(train_expected, train_predicted)))
print("Result:\n%s" % np.sqrt(mean_squared_error(expected, predicted)))

Training:
1.03201521992
Result:
1.03783474054


### Computed Ratings on missing values

In [89]:
missing_test = test_rdd.subtractByKey(test)

In [90]:
missing_ratings_user = missing_test.map(lambda x: ((x[0][0]), (x[0][1], x[1]))).join(avg_rating).map(lambda x: ((x[0], x[1][0][0]), (x[1][1], x[1][0][1])))

In [91]:
missing_ratings_user.take(5)

[(('70sSlkooEgL_TEjWDQbr3A', 'afXMX5llxcMFzbaPaBBm6A'), (3.12, 4.0)),
 (('pr8_C12oHakeNB4ZPp_dig', '3ZcxnR9YkDVRqqkDJMRWBg'), (4.2, 5.0)),
 (('dz8CFWEWuR_4S1zlZhWCMQ', 'rKh_Nl5edIB9AevqnDmO6g'),
  (3.7291666666666665, 3.0)),
 (('dz8CFWEWuR_4S1zlZhWCMQ', '_VYUU5HPLYasd-xdKLimNA'),
  (3.7291666666666665, 3.0)),
 (('z4MQzyewTRzSoStg0NwL-w', 'lvZOJWiwNymeBhOAgoy11w'),
  (3.911764705882353, 4.0))]

Make predictions on test data using the trained regressor

In [92]:
predictions = test.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [93]:
final_predictions = predictions.union(missing_ratings_user)

In [68]:
final_predictions.count()

12496

In [69]:
mse = final_predictions.map(lambda x: (x[1][0]-x[1][1])**2)

In [70]:
np.sqrt(mse.mean())

1.0307912822424812

In [94]:
final_predictions = final_predictions.collect()

In [95]:
final_predictions

[((u'nzl2KyGKLtV7j8QEfOhC-w', u'TU2oECyx4VhCnzRzGzOlTg'),
  (3.8320487251520476, 4.0)),
 ((u'JeIVRtZiwUnSP9qR4AmfEQ', u'WZDQqw960DaMWDyPbtRFtA'),
  (3.8320487251520476, 4.0)),
 ((u'g3V76Ja0XgWS1rqx0gxL_A', u'a384anuGRsOe5IXclb3cNQ'),
  (3.8301495633074758, 4.0)),
 ((u'3DltS5Wr9MOntnc8StBFBA', u'GjX2WgchvvS-JAWBmzkLyQ'),
  (3.8301495633074758, 5.0)),
 ((u'GMKoemATfrXg1deaXxt2jA', u'gldPX9ANF5Nic0N7igu2og'),
  (3.8301495633074758, 5.0)),
 ((u'ACwBMSJzgW6vOvV7vOrk8Q', u'ndtSMHlCSKDtEBoOYAdL1Q'),
  (3.8301495633074758, 2.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'rOHLBGBcWzlxdvNI6squgQ'),
  (3.8301495633074758, 4.0)),
 ((u'mAOD3ynoKuxI6gqLP8k7qw', u'17DImn_vPnuwU9hhRFXlUA'),
  (3.8320487251520476, 5.0)),
 ((u'Jk0AcqPCVMfc_DTWO3SzwA', u'eNsM5ezjhPQWI6XlUGrUhw'),
  (3.8301495633074758, 5.0)),
 ((u'3oKtLdAO138VHY8KTlpXuQ', u'O40nlfiMpWAKgMx7bH9h6Q'),
  (3.8320487251520476, 5.0)),
 ((u'MLf4JUa1uzQkDLrBJ5yJHA', u'K8M3OeFCcAnxuxtTc0BQrQ'),
  (3.8301495633074758, 4.0)),
 ((u'YNkbvkPhLMFlOFYH2PLMwA', u'

### Save predictions file

In [96]:
with open('/Users/lakshya/Desktop/INF-553/Project/Doc2Vec_TextBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in final_predictions:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")

### Making predictions on training data for Hybrid model

In [99]:
train_predict = train.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [100]:
train_predict.take(5)

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (3.8301495633074758, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (3.8301495633074758, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (3.8301495633074758, 3.0)),
 ((u'M9MXoSsb193m1g0QmmUYBQ', u'-i3pCgQi_Y9NiSSWs6G7bw'),
  (3.8301495633074758, 5.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'x8WI_GkeGHGJCXggDm8flg'),
  (3.8301495633074758, 3.0))]

In [73]:
train_predictions = train_predict.map(lambda x: (x[1][0]-x[1][1])**2)

In [74]:
np.sqrt(train_predictions.mean())

1.0323727636048268

In [101]:
train_predict = train_predict.collect()

In [102]:
with open('/Users/lakshya/Desktop/INF-553/Project/TrainDoc2Vec_TextBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in train_predict:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")